In [2]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
from cmath import log

In [3]:
iris=load_iris()
data=pd.DataFrame(iris.data,columns=["sl", "sw", 'pl', 'pw'])
print(data)

      sl   sw   pl   pw
0    5.1  3.5  1.4  0.2
1    4.9  3.0  1.4  0.2
2    4.7  3.2  1.3  0.2
3    4.6  3.1  1.5  0.2
4    5.0  3.6  1.4  0.2
..   ...  ...  ...  ...
145  6.7  3.0  5.2  2.3
146  6.3  2.5  5.0  1.9
147  6.5  3.0  5.2  2.0
148  6.2  3.4  5.4  2.3
149  5.9  3.0  5.1  1.8

[150 rows x 4 columns]


In [4]:
def label(val, *boundaries):
    if (val < boundaries[0]):
        return 'a'
    elif (val < boundaries[1]):
        return 'b'
    elif (val < boundaries[2]):
        return 'c'
    else:
        return 'd'

def toLabel(data, old_feature_name):
    second = data[old_feature_name].mean()
    minimum = data[old_feature_name].min()
    first = (minimum + second)/2
    maximum = data[old_feature_name].max()
    third = (maximum + second)/2
    return data[old_feature_name].apply(label, args= (first, second, third))

data['sl_labeled'] = toLabel(data, 'sl')
data['sw_labeled'] = toLabel(data, 'sw')
data['pl_labeled'] = toLabel(data, 'pl')
data['pw_labeled'] = toLabel(data, 'pw')

data.drop(['sl', 'sw', 'pl', 'pw'], axis = 1, inplace = True)



In [5]:
remaining_f=data.columns
print(remaining_f)
data["result"]=iris.target

Index(['sl_labeled', 'sw_labeled', 'pl_labeled', 'pw_labeled'], dtype='object')


In [6]:
data

,sl_labeled,sw_labeled,pl_labeled,pw_labeled,result
0,b,c,a,a,0
1,a,b,a,a,0
2,a,c,a,a,0
3,a,c,a,a,0
4,a,c,a,a,0
...,...,...,...,...,...
145,c,b,c,d,2
146,c,a,c,d,2
147,c,b,c,d,2
148,c,c,d,d,2


In [7]:
def info(arr=[]): #arr is y
    m=0
    s=len(arr)
    arr=np.array(arr)
    e,c=np.unique(arr,return_counts=True)
    l=len(e)
    for i in range(l):
        m-=(c[i]/s)*log(c[i]/s)
    return m

In [8]:
def gain(data,f):
    D=data.shape
    D=D[0]
    info_f=0
    info_org=info(data.result)
    splitinfo=0
    h=set(data[f])
    for i in h:
        k=data[data[f]==i]
        Di=k.shape
        Di=Di[0]
        v=[]
        n=0
        info_f+=(info(k.result))*abs((Di/D))
        splitinfo-=abs((Di/D))*log(abs((Di/D)))
    info_gain=info_org-info_f
    gainratio=info_gain/splitinfo
    return gainratio

In [9]:
def build_tree(data,remaining_f,level,c_class):
    if(len(set(data.result))==1 or len(remaining_f)==0):
        return 0
    best_s=0
    best_f=0
    n=0
    print("")
    for i in remaining_f: #np.nditer(remaining_f,):
        s=gain(data,i)
        if(best_s<s):
            best_s=s
            best_f=n
        n+=1 
    print("LEVEL ",level)
    if(level==1):
        print("Current Class=",remaining_f[best_f])
    else:
        print("Current Class=",c_class)
    print("Best Feature=",remaining_f[best_f])
    print("Gain Ratio=",best_s)
    k=remaining_f[best_f]
    j=k
    e=np.unique(data[remaining_f[best_f]],return_counts=False)
    remaining_f=np.delete(remaining_f,best_f)  
    k=np.array(k)
    l=len(e)
    for i in range(l):
        h=j
        datai=data.drop(data[data[k]==e[i]].index)
        h=h+" "+e[i]
        build_tree(datai,remaining_f,(level+1),h)

In [10]:
build_tree(data,remaining_f,1,"")


LEVEL  1
Current Class= pw_labeled
Best Feature= pw_labeled
Gain Ratio= (0.6996382036222091+0j)

LEVEL  2
Current Class= pw_labeled a
Best Feature= pl_labeled
Gain Ratio= (0.3571407602069344+0j)

LEVEL  3
Current Class= pl_labeled b
Best Feature= sl_labeled
Gain Ratio= (0.10269466555655302+0j)

LEVEL  4
Current Class= sl_labeled a
Best Feature= sw_labeled
Gain Ratio= (0.032092593520980914+0j)

LEVEL  4
Current Class= sl_labeled b
Best Feature= sw_labeled
Gain Ratio= (0.01651114503773901+0j)

LEVEL  4
Current Class= sl_labeled c
Best Feature= sw_labeled
Gain Ratio= (0.05711352972366223+0j)

LEVEL  4
Current Class= sl_labeled d
Best Feature= sw_labeled
Gain Ratio= (0.016187311636752525+0j)

LEVEL  3
Current Class= pl_labeled c
Best Feature= sl_labeled
Gain Ratio= (0.412996969675857+0j)

LEVEL  4
Current Class= sl_labeled a
Best Feature= sw_labeled
Gain Ratio= (0.11410062391098309+0j)

LEVEL  4
Current Class= sl_labeled b
Best Feature= sw_labeled
Gain Ratio= (0.2052855139982127+0j)

LEVE